In [12]:
from classes import *
import numpy as np
import networkx as nx
import pylab as plt
from scipy.stats import norm,uniform,binom,poisson
import random
from functools import reduce
from itertools import product
from collections import deque
import re
from time import sleep
import json

In [2]:
import requests as req
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from webdriver_manager.firefox import GeckoDriverManager

PATH=FirefoxService(GeckoDriverManager().install())
binary = FirefoxBinary(r'C:\Program Files\Mozilla Firefox\Firefox.exe')

opciones=Options()

[WDM] - Downloading: 16.9kB [00:00, 921kB/s]                    
[WDM] - Downloading: 16.9kB [00:00, 23.1MB/s]                   
[WDM] - Downloading: 100%|██████████| 1.59M/1.59M [00:07<00:00, 228kB/s] 


In [4]:
line_dict = {}

def helper_fn(element):
    t = zip([j.get_attribute('title') for i in element.find_elements(By.XPATH, 'tbody/tr') for j in i.find_elements(By.XPATH, 'td/a')],[j.text for i in element.find_elements(By.XPATH, 'tbody/tr') for j in i.find_elements(By.XPATH, 'td/a')])
    delete = {('Cercanías Madrid', ''),('Mantenimiento', '')}
    t = filter(lambda x: x not in delete, t)
    t = list(map(lambda x: re.search(r'\d+',x[0]).group(0) if bool(re.search(r'\d+',x[0])) else x[1] , t))
    s = []
    for i in t:
        try:
            int(i)
        except:
            s.append((i,[]))
        else:
            s[-1][1].append(int(i))
    return s

def table_extractor(k):
    driver = webdriver.Firefox(firefox_binary=binary,options = opciones)
    url = 'https://es.wikipedia.org/wiki/L%C3%ADnea_'+ str(k) +'_(Metro_de_Madrid)'
    xpath1 = '/html/body/div[3]/div[3]/div[5]/div[1]/table[2]'
    xpath2 = '/html/body/div[3]/div[3]/div[5]/div[1]/table[3]'
    driver.get(url)
    try:
        x1 = helper_fn(driver.find_element(By.XPATH, xpath1))
    except:
        x1 = []
    try:
        x2 = helper_fn(driver.find_element(By.XPATH, xpath2))
    except:
        x2 = []
    driver.quit()
    return [x1,x2]

In [5]:
for i in range(1,11):
    line_dict[i] = table_extractor(i)

C:\Users\jorge\AppData\Local\Temp\ipykernel_16684\3445140817.py:19: DeprecationWarning: firefox_binary has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(firefox_binary=binary,options = opciones)


In [30]:
line_dict_clean = {
    1: line_dict[1][0],
    2: line_dict[2][0],
    3: line_dict[3][0],
    4: line_dict[4][1],
    5: line_dict[5][1],
    6: line_dict[6][0],
    7: line_dict[7][1],
    # method has an error for line 8
    9: line_dict[9][1],
    10: line_dict[10][0],
}

In [31]:
#### Special Case for line 8
driver = webdriver.Firefox(firefox_binary=binary,options = opciones)
url = 'https://es.wikipedia.org/wiki/L%C3%ADnea_8_(Metro_de_Madrid)'
xpath2 = '/html/body/div[3]/div[3]/div[5]/div[1]/table[3]'
driver.get(url)
x = driver.find_element(By.XPATH, xpath2)
y = zip([j.get_attribute('title') for i in x.find_elements(By.XPATH, 'tbody/tr') for j in i.find_elements(By.XPATH, 'td/a')],[j.text for i in x.find_elements(By.XPATH, 'tbody/tr') for j in i.find_elements(By.XPATH, 'td/a')])
delete = {('Cercanías Madrid', ''),('Mantenimiento', '')}
t = list(filter(lambda x: x not in delete, y))
r = list(map( lambda x : (x[1],[]) ,t[:4] ))
t = t[4:]
t = list(map(lambda x: re.search(r'\d+',x[0]).group(0) if bool(re.search(r'\d+',x[0])) else x[1] , t))
s = []
for i in t:
    try:
        int(i)
    except:
        s.append((i,[]))
    else:
        s[-1][1].append(int(i))
line_dict_clean[8] = r + s
driver.quit()

C:\Users\jorge\AppData\Local\Temp\ipykernel_16684\3602081523.py:2: DeprecationWarning: firefox_binary has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(firefox_binary=binary,options = opciones)


In [38]:
for keys, values in line_dict_clean.items():
    line_dict_clean[keys] = list(filter(lambda x: x[0],values))
line_dict_clean_copy = {key: value[:] for key, value in line_dict_clean.items()}
with open('../data/metromadrid/dict_of_lines.json', 'w') as fp: # backup
    json.dump(line_dict_clean_copy, fp)
with open('dict_of_lines.json', 'w') as fp: # backup
    json.dump(line_dict_clean_copy, fp)

In [41]:
with open('../data/metromadrid/dict_of_lines.json', 'r') as fp: # Load path for top 10 song data for each artist in our dataset
    line_dict_clean = json.load(fp)
    line_dict_clean = {int(key): value for key, value in line_dict_clean.items()}


In [92]:
line_node_paths = {}
for k in range(1,11):
    line_node_paths[k] = []
    count = 0
    for i,j in enumerate(line_dict_clean[k]):
        if j[1] or i == 0 or i == len(line_dict_clean[k])-1:
            if j[1]:
                if j[1][0] >= k:
                    line_node_paths[k].append((k,i,count))
                else:
                    val = reduce(lambda x,y : x if x[0] == j[0] else y,line_dict_clean[j[1][0]])
                    line_node_paths[k].append((j[1][0],line_dict_clean[j[1][0]].index(val),count))
            else:
                line_node_paths[k].append((k,i,count))
            count = 0
        count += 1

nodes = []
edges=  []

for i in range(1,11):
    path = line_node_paths[i]
    
    for j in range(len(path)-1):
        node1 = Node((path[j][0],path[j][1]))
        node2 = Node((path[j+1][0],path[j+1][1]))
        if node1 not in nodes:
            nodes.append(node1)
        if node2 not in nodes:
            nodes.append(node2)
        edges.append(Edge((node1,node2),weight=path[j+1][2]))

node_translation = {}
for node in nodes:
    if node not in node_translation:
        node_translation[node] = line_dict_clean[node[0]][node[1]][0]
node_reverse_translation = {v: k for k, v in node_translation.items()}
network = Network(nodes,edges)

In [114]:
x = network.find_path(node_reverse_translation['Legazpi'],node_reverse_translation['Cuatro Caminos'])
stations = 0
for j,i in enumerate(x):
    print(node_translation[i])
    if j < len(x)-1:

        edge = network.edge_adjacency[i]
        for k in edge:
            if x[j+1] in k:
                stations += k.weight
print(stations)

Legazpi
Plaza Elíptica
Oporto
Casa de Campo
Cuatro Caminos
17


In [98]:
node_reverse_translation

{'Pinar de Chamartín': Node('1.00', '0.00'),
 'Chamartín': Node('1.00', '3.00'),
 'Plaza de Castilla': Node('1.00', '5.00'),
 'Cuatro Caminos': Node('2.00', '26.00'),
 'Bilbao': Node('1.00', '15.00'),
 'Tribunal': Node('1.00', '16.00'),
 'Gran Vía': Node('1.00', '18.00'),
 'Sol': Node('1.00', '19.00'),
 'Atocha': Node('1.00', '23.00'),
 'Pacífico': Node('1.00', '26.00'),
 'Valdecarros': Node('1.00', '40.00'),
 'Las Rosas': Node('2.00', '0.00'),
 'Ventas': Node('2.00', '7.00'),
 'Manuel Becerra': Node('2.00', '9.00'),
 'Goya': Node('2.00', '10.00'),
 'Príncipe de Vergara': Node('2.00', '11.00'),
 'Ópera': Node('2.00', '19.00'),
 'Noviciado': Node('2.00', '21.00'),
 'San Bernardo': Node('2.00', '22.00'),
 'Canal': Node('2.00', '25.00'),
 'Moncloa': Node('3.00', '0.00'),
 'Argüelles': Node('3.00', '2.00'),
 'Callao': Node('3.00', '5.00'),
 'Embajadores': Node('3.00', '9.00'),
 'Palos de la Frontera': Node('3.00', '10.00'),
 'Legazpi': Node('3.00', '14.00'),
 'Usera': Node('3.00', '16.00')

58